In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200612183630-0002
KERNEL_ID = 4657258a-5c60-4c07-8183-5dd61e5f3800


In [2]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_bbb9d49e16004c7db512036badb7fbad = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='D8tizBwStn6rXRlAtXde-MAi6Yc89xEB574Hnsst0l3S',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_bbb9d49e16004c7db512036badb7fbad.get_object(Bucket='watsonmlsiinternship-donotdelete-pr-ybl7jdnawpjicy',Key='50_Startups.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()


,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
dataset.isnull().sum()

R&D Spend          0
Administration     0
Marketing Spend    0
State              0
Profit             0
dtype: int64

In [4]:
state = pd.get_dummies(dataset['State'], drop_first= True)
dataset.drop('State', axis = 1, inplace = True)
dataset = pd.concat([state, dataset], axis = 1)

In [5]:
dataset.head()

,Florida,New York,R&D Spend,Administration,Marketing Spend,Profit
0,0,1,165349.20,136897.80,471784.10,192261.83
1,0,0,162597.70,151377.59,443898.53,191792.06
2,1,0,153441.51,101145.55,407934.54,191050.39
3,0,1,144372.41,118671.85,383199.62,182901.99
4,1,0,142107.34,91391.77,366168.42,166187.94


In [6]:
dataset.describe()

,Florida,New York,R&D Spend,Administration,Marketing Spend,Profit
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.320000,0.340000,73721.615600,121344.639600,211025.097800,112012.639200
std,0.471212,0.478518,45902.256482,28017.802755,122290.310726,40306.180338
min,0.000000,0.000000,0.000000,51283.140000,0.000000,14681.400000
25%,0.000000,0.000000,39936.370000,103730.875000,129300.132500,90138.902500
50%,0.000000,0.000000,73051.080000,122699.795000,212716.240000,107978.190000
75%,1.000000,1.000000,101602.800000,144842.180000,299469.085000,139765.977500
max,1.000000,1.000000,165349.200000,182645.560000,471784.100000,192261.830000


In [7]:
dataset.corr()

,Florida,New York,R&D Spend,Administration,Marketing Spend,Profit
Florida,1.000000,-0.492366,0.105711,0.010493,0.205685,0.116244
New York,-0.492366,1.000000,0.039068,0.005145,-0.033670,0.031368
R&D Spend,0.105711,0.039068,1.000000,0.241955,0.724248,0.972900
Administration,0.010493,0.005145,0.241955,1.000000,-0.032154,0.200717
Marketing Spend,0.205685,-0.033670,0.724248,-0.032154,1.000000,0.747766
Profit,0.116244,0.031368,0.972900,0.200717,0.747766,1.000000


In [8]:
x = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [9]:
from sklearn.model_selection import train_test_split as split
x_train, x_test, y_train, y_test = split(x, y, test_size = 0.2, random_state = 0)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [11]:
from sklearn.decomposition import KernelPCA
kpca = KernelPCA(n_components = 2, kernel = 'rbf')
x_train = kpca.fit_transform(x_train)
x_test = kpca.transform(x_test)

In [12]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
y_pred = regressor.predict(x_test)

In [14]:
np.set_printoptions(precision = 2)
print(np.concatenate((y_test.reshape(len(y_test), 1), y_pred.reshape(len(y_pred), 1)), 1))

[[103282.38 112195.34]
 [144259.4  104223.24]
 [146121.95 122039.87]
 [ 77798.83 104431.71]
 [191050.39 127186.61]
 [105008.31 149387.1 ]
 [ 81229.06 106223.78]
 [ 97483.56 110694.15]
 [110352.25 122848.68]
 [166187.94 125510.23]]


In [15]:
regressor.score(x_test, y_test)

0.09063885963709994